In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import os
from pathlib import Path
from tqdm import tqdm
import pdfplumber

In [60]:
OEP = "https://web.oep.org.bo"
LISTA = "listas/2014/habilitados.pdf"

In [61]:
def descargar_lista(n_partidos=8):
    os.makedirs(LISTAS, exist_ok=True)
    url = f"{OEP}/wp-content/uploads/2017/01/lista_de_candidatos_habilitados_2014.pdf"
    path = LISTA
    with open(path, "wb") as f:
        response = requests.get(url)
        f.write(response.content)

In [62]:
descargar_lista()

In [63]:
def extraer_datos():
    pdf = pdfplumber.open(LISTA)
    ornaments = ["Elecciones Generales 2014", "LISTA DE CANDIDATAS (OS) HABILITADOS"]
    data = []
    for page in tqdm(pdf.pages):
        for line in page.extract_text().split("\n"):
            if line not in ornaments:
                try:
                    if re.search("^Organización Política", line):
                        partido = line.split(":")[-1].strip()
                    if re.search("^Sigla", line):
                        sigla = line.split(":")[-1].strip()
                    if re.search("^Candidatura", line):
                        candidatura = line.split(":")[-1].strip()
                    if re.search("^País", line):
                        pais = line.split(":")[-1].strip()
                        departamento = ""
                    if re.search("^Departamento", line):
                        departamento = line.split(":")[-1].strip()
                    if re.search("^Posición", line):
                        header = line
                    if re.search("^[0-9]+", line):
                        words = line.split()
                        posicion, titularidad, nro_documento, edad, sexo = [
                            words[i] for i in [0, 1, -3, -2, -1]
                        ]
                        nombre = " ".join(words[2:-3])
                        data.append({
                            "partido": partido,
                            "sigla": sigla,
                            "candidatura": candidatura,
                            "departamento": departamento,
                            "posición": posicion,
                            "titularidad": titularidad,
                            "nombre_completo": nombre,
                            "nro_documento": nro_documento,
                            "sexo": sexo,
                            "edad": edad
                        })
                except Exception as e:
                    print(f"{e}: {line}")
    return pd.DataFrame(data)

In [64]:
df = extraer_datos()

100%|██████████| 43/43 [00:02<00:00, 15.88it/s]


In [65]:
df

,partido,sigla,candidatura,departamento,posición,titularidad,nombre_completo,nro_documento,sexo,edad
0,MOVIMIENTO AL SOCIALISMO INSTRUMENTO POLITICO ...,MAS-IPSP,Presidenta - Presidente,,1,Titular,JUAN EVO MORALES AYMA,2763595,M,55
1,MOVIMIENTO AL SOCIALISMO INSTRUMENTO POLITICO ...,MAS-IPSP,Vicepresidenta - Vicepresidente,,1,Titular,ALVARO MARCELO GARCIA LINERA,3007962,M,52
2,MOVIMIENTO AL SOCIALISMO INSTRUMENTO POLITICO ...,MAS-IPSP,Senadoras - Senadores,Chuquisaca,1,Titular,LEONIDAS MILTON BARON HIDALGO,3651095,M,39
3,MOVIMIENTO AL SOCIALISMO INSTRUMENTO POLITICO ...,MAS-IPSP,Senadoras - Senadores,Chuquisaca,1,Suplente,DELINA CUMANDIRI ROMERO,6247284,F,37
4,MOVIMIENTO AL SOCIALISMO INSTRUMENTO POLITICO ...,MAS-IPSP,Senadoras - Senadores,Chuquisaca,2,Titular,NELIDA SIFUENTES CUETO,5641588,F,33
...,...,...,...,...,...,...,...,...,...,...
1038,UNIDAD DEMOCRATA,UD,Representantes SupraEstatales,Santa Cruz,2,Suplente,FABIOLA ZAMBRANA PARDO,5343647,F,23
1039,UNIDAD DEMOCRATA,UD,Representantes SupraEstatales,Beni,3,Titular,MERY ELINA ZABALA MONTENEGRO,1725934,F,50
1040,UNIDAD DEMOCRATA,UD,Representantes SupraEstatales,Beni,3,Suplente,ALVARO EDUARDO COIMBRA CORNEJO,4186833,M,33
1041,UNIDAD DEMOCRATA,UD,Representantes SupraEstatales,Pando,9,Titular,LOURDES MACHACA GUANCA,4201299,F,37


In [66]:
df.to_parquet("datos/2014.parquet")